In [16]:
import pandas as pd

raw_data_location='s3://anomaly-detection-team-vypin/PS_20174392719_1491204439457_log.csv'
output_data_location_random_shuffled='s3://anomaly-detection-team-vypin/data.csv'

paysim_df=pd.read_csv(raw_data_location)
paysim_df.head()




,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [17]:
paysim_df.dtypes


step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [18]:
paysim_df_reduced=paysim_df.drop(columns=['nameDest','nameOrig'])
paysim_df_reduced.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,0
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [19]:
paysim_df_hotencoded=pd.get_dummies(paysim_df_reduced)

In [20]:
paysim_df_hotencoded.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,170136.0,160296.36,0.0,0.0,0,0,0,0,0,1,0
1,1,1864.28,21249.0,19384.72,0.0,0.0,0,0,0,0,0,1,0
2,1,181.00,181.0,0.00,0.0,0.0,1,0,0,0,0,0,1
3,1,181.00,181.0,0.00,21182.0,0.0,1,0,0,1,0,0,0
4,1,11668.14,41554.0,29885.86,0.0,0.0,0,0,0,0,0,1,0


In [21]:
from sklearn.utils import shuffle
paysim_df_shuffled=shuffle(paysim_df_hotencoded)

In [22]:
paysim_df_shuffled.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
3194246,249,13943.45,662.00,0.00,0.00,0.00,0,0,0,0,0,1,0
2133730,183,3407.11,388797.11,385390.00,0.00,0.00,0,0,0,0,0,1,0
2924368,229,192322.43,94004.00,0.00,216766.67,409089.11,0,0,0,1,0,0,0
2716278,211,11709.31,16266.00,4556.69,0.00,0.00,0,0,0,0,0,1,0
4145971,303,6483.12,0.00,0.00,0.00,0.00,0,0,0,0,0,1,0


In [23]:
output_data_location_random_shuffled='./data.csv'
paysim_df_shuffled.insert(0,'label',paysim_df_shuffled['isFraud'])



In [24]:
paysim_df_shuffled.groupby('label').count()
paysim_df_shuffled.drop(columns=['isFraud'])

,label,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
3194246,0,249,13943.45,662.00,0.00,0.00,0.00,0,0,0,0,1,0
2133730,0,183,3407.11,388797.11,385390.00,0.00,0.00,0,0,0,0,1,0
2924368,0,229,192322.43,94004.00,0.00,216766.67,409089.11,0,0,1,0,0,0
2716278,0,211,11709.31,16266.00,4556.69,0.00,0.00,0,0,0,0,1,0
4145971,0,303,6483.12,0.00,0.00,0.00,0.00,0,0,0,0,1,0
2085784,0,182,162551.08,0.00,0.00,1432458.96,1595010.05,0,0,0,0,0,1
3767071,0,280,370872.01,130961.47,0.00,9378544.94,9749416.95,0,0,1,0,0,0
4395582,0,321,49548.77,1760.00,51308.77,0.00,0.00,0,1,0,0,0,0
384854,0,17,383468.09,145822.39,0.00,0.00,383468.09,0,0,1,0,0,0
4147457,0,303,50439.25,34502.00,0.00,0.00,50439.25,0,0,1,0,0,0


In [25]:
paysim_df_shuffled.to_csv(output_data_location_random_shuffled)

In [4]:
import boto3

#Put the prepped file to S3
s3 = boto3.client('s3')
s3.upload_file('./data.csv', 'anomaly-detection-team-vypin', 'vishal/data.csv')